In [4]:
from collections import OrderedDict
import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from base import BaseModel
import numpy as np
from xml.etree import ElementTree

In [5]:
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, x):
        N = x.data.size(0)
        C = x.data.size(1)
        H = x.data.size(2)
        W = x.data.size(3)
        x = F.avg_pool2d(x, (H, W))
        x = x.view(N, C)
        return x

In [44]:
class Darknet19(BaseModel):
    def __init__(self, pretrained=True):
        super(Darknet19, self).__init__()

        self.features = nn.Sequential(OrderedDict([
        ('layer1', nn.Sequential(OrderedDict([
            ('conv1_1', nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn1_1', nn.BatchNorm2d(32)),
            ('leaky1_1', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool1', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer2', nn.Sequential(OrderedDict([
            ('conv2_1', nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn2_1', nn.BatchNorm2d(64)),
            ('leaky2_1', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool2', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer3', nn.Sequential(OrderedDict([
            ('conv3_1', nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn3_1', nn.BatchNorm2d(128)),
            ('leaky3_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv3_2', nn.Conv2d(128, 64, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn3_2', nn.BatchNorm2d(64)),
            ('leaky3_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv3_3', nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn3_3', nn.BatchNorm2d(128)),
            ('leaky3_3', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool3', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer4', nn.Sequential(OrderedDict([
            ('conv4_1', nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn4_1', nn.BatchNorm2d(256)),
            ('leaky4_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv4_2', nn.Conv2d(256, 128, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn4_2', nn.BatchNorm2d(128)),
            ('leaky4_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv4_3', nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn4_3', nn.BatchNorm2d(256)),
            ('leaky4_3', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool4', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer5', nn.Sequential(OrderedDict([
            ('conv5_1', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_1', nn.BatchNorm2d(512)),
            ('leaky5_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_2', nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn5_2', nn.BatchNorm2d(256)),
            ('leaky5_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_3', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_3', nn.BatchNorm2d(512)),
            ('leaky5_3', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_4', nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=1, bias=False)),
            ('bn5_4', nn.BatchNorm2d(256)),
            ('leaky5_4', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_5', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_5', nn.BatchNorm2d(512)),
            ('leaky5_5', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool5', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer6', nn.Sequential(OrderedDict([
            ('conv6_1', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_1', nn.BatchNorm2d(1024)),
            ('leaky6_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_2', nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn6_2', nn.BatchNorm2d(512)),
            ('leaky6_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_3', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_3', nn.BatchNorm2d(1024)),
            ('leaky6_3', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_4', nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=1, bias=False)),
            ('bn6_4', nn.BatchNorm2d(512)),
            ('leaky6_4', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_5', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_5', nn.BatchNorm2d(1024)),
            ('leaky6_5', nn.LeakyReLU(0.1, inplace=True))
        ])))
        ]))

        self.classifier = nn.Sequential(OrderedDict([
        ('classifier', nn.Sequential(OrderedDict([
        ('conv7_1', nn.Conv2d(1024, 1000, kernel_size=(1, 1), stride=(1, 1))),
        ('globalavgpool', GlobalAvgPool2d()),
        ('softmax', nn.Softmax(dim=1))
        ])))]))

        if pretrained:
            # self.load_state_dict(model_zoo.load_url(model_paths['darknet19'],  progress=True))
            self.load_weight()
            print('Model is loaded')

    def forward(self, x):
        out = self.features(x)
        out = self.classifier(out)
        return out
    def load_weight(self):
        weight_file = 'weights/darknet19-deepBakSu-e1b3ec1e.pth'
        assert len(torch.load(weight_file).keys()) == len(self.state_dict().keys())
        dic = {}
        for now_keys, values in zip(self.state_dict().keys(), torch.load(weight_file).values()):
            dic[now_keys]=values
        self.load_state_dict(dic)
        print('Weights are loaded!')

In [45]:
model = Darknet19(True)

Weights are loaded!
Model is loaded


In [8]:
def make_directory(base_path : str) -> int :
    """
        Checks if a directory exists and if doesn't creates the directory.

        Args:
        base_path : Directory path which will be created if it doesn't exist.

        Returns 0 if directory exists else 1
    """
    if os.path.exists(base_path) :
        return 0

    # Create the directory since the path doesn't exist.
    os.mkdir(base_path)
    if os.path.exists(base_path) :
        return 0

    # Path doesn't exist as well as directory couldn't be created.
    print("Error : Cannot create desired path : ", base_path)
    return 1

In [9]:
def generate_csv(csv_name : str, weight_matrix : torch.tensor, base_path : str) -> str :
    """
        Generates csv for weights or bias matrix.

        Args:
        csv_name : A string name for csv file which will store the weights.
        weight_matrix : A torch tensor holding weights that will be stored in the matrix.
        base_path : Base path where csv will be stored.
    """
    # Check if base path exists else create directory.
    make_directory(base_path)
    file_path = os.path.join(base_path, csv_name)
    np.savetxt(file_path, weight_matrix.numpy().ravel())
    return file_path

In [10]:
def extract_weights(layer, layer_index, base_path) -> dict :
    """
        Extracts weights, biases and other parameters required to reproduce
        the same output.

        Args:
        layer : An torch.nn object (layer).
        layer_index : A string determining name of csv file that will be appended to
                      name of layer.
                      Eg. if layer = nn.Conv2d and layer_index = 0
                          csv_filename = Conv_layer_index.csv
        base_path : A string depicting base path for storing weight / bias csv.

        Returns dictionary of parameter description and parameters.

        Exceptions:
        Currently this has only been tested for convolutional and batch-norm layer.
    """
    parameter_dictionary = {}
    if isinstance(layer, nn.Conv2d):
        # The layer corresponds to Convolutional layer.
        # For convolution layer we require weights and biases to reproduce the
        # same result.
        parameter_dictionary["name"] = "Convolution2D"
        parameter_dictionary["input-channels"] = layer.in_channels
        parameter_dictionary["output-channels"] = layer.out_channels
        # Assume weight matrix is never empty for nn.Conv2d()
        parameter_dictionary["has_weights"] = 1
        parameter_dictionary["weight_offset"] = 0
        csv_name = "conv_weight_" + layer_index + ".csv"
        parameter_dictionary["weight_csv"] = generate_csv(csv_name, \
            layer.weight.detach(), base_path)
        if layer.bias != None:
            parameter_dictionary["has_bias"] = 1
            parameter_dictionary["bias_offset"] = 0
            bias_csv_name = "conv_bias_" + layer_index + ".csv"
            parameter_dictionary["bias_csv"] = generate_csv(bias_csv_name, \
                layer.bias.detach(), base_path)
        else:
            parameter_dictionary["has_bias"] = 0
            parameter_dictionary["bias_offset"] = layer.out_channels
            parameter_dictionary["bias_csv"] = "None"
        parameter_dictionary["has_running_mean"] = 0
        parameter_dictionary["running_mean_csv"] = "None"
        parameter_dictionary["has_running_var"] = 0
        parameter_dictionary["running_var_csv"] = "None"
    elif isinstance(layer, nn.BatchNorm2d) :
        # The layer corresponds to Batch Normalization layer.
        # For batchnorm layer we require weights, biases and running mean and running variance
        # to reproduce the same result.
        parameter_dictionary["name"] = "BatchNorm2D"
        parameter_dictionary["input-channels"] = layer.num_features
        parameter_dictionary["output-channels"] = layer.num_features
        # Assume weight matrix is never empty for nn.BatchNorm2d()
        parameter_dictionary["has_weights"] = 1
        parameter_dictionary["weight_offset"] = 0
        csv_name = "batchnorm_weight_" + layer_index + ".csv"
        parameter_dictionary["weight_csv"] = generate_csv(csv_name, \
            layer.weight.detach(), base_path)
        if layer.bias != None:
            parameter_dictionary["has_bias"] = 1
            parameter_dictionary["bias_offset"] = 0
            bias_csv_name = "batchnorm_bias_" + layer_index + ".csv"
            parameter_dictionary["bias_csv"] = generate_csv(bias_csv_name, \
                layer.bias.detach(), base_path)
        else:
            parameter_dictionary["has_bias"] = 0
            parameter_dictionary["bias_offset"] = layer.out_channels
            parameter_dictionary["bias_csv"] = "None"
        # Assume BatchNorm layer always running variance and running mean.
        running_mean_csv = "batchnorm_running_mean_" + layer_index + ".csv"
        parameter_dictionary["has_running_mean"] = 1
        parameter_dictionary["running_mean_csv"] = generate_csv(running_mean_csv, \
            layer.running_mean.detach(), base_path)
        parameter_dictionary["has_running_var"] = 1
        running_var_csv = "batchnorm_running_var_" + layer_index + ".csv" 
        parameter_dictionary["running_var_csv"] = generate_csv(running_mean_csv, \
            layer.running_var.detach(), base_path)
    else :
        # The layer corresponds to un-supported layer or layer doesn't have trainable
        # parameter. Example of such layers are nn.MaxPooling2d() and nn.SoftMax.
        parameter_dictionary["name"] = "unknown_layer"
        parameter_dictionary["input-channels"] = 0
        parameter_dictionary["output-channels"] = 0
        parameter_dictionary["has_weights"] = 0
        parameter_dictionary["weight_offset"] = 0
        parameter_dictionary["weight_csv"] = "None"
        parameter_dictionary["has_bias"] = 0
        parameter_dictionary["bias_offset"] = 0
        parameter_dictionary["bias_csv"] = "None"
        parameter_dictionary["has_running_mean"] = 0
        parameter_dictionary["running_mean_csv"] = "None"
        parameter_dictionary["has_running_var"] = 0
        parameter_dictionary["running_var_csv"] = "None"
    return parameter_dictionary

In [65]:
def create_xml_tree(parameter_dictionary : dict, root_tag = "layer") -> ElementTree.ElementTree() :
    """
        Creates an XML tree from a dictionary wrapped around root tag.

        Args:
        parameter_dictionary : Dictionary which will be converted to xml tree.
        root_tag : Tag around which elements of dictionary will be wrapped.
                    Defaults to "layer".
    
        Returns : ElementTree.ElementTree() object.
    """
    layer = ElementTree.Element(root_tag)
    for parameter_desc in parameter_dictionary :
        parameter_description = ElementTree.Element(parameter_desc)
        parameter_description.text = str(parameter_dictionary[parameter_desc])
        layer.append(parameter_description)
    return layer

In [70]:
def create_xml_file(parameter_dictionary : dict,
                    xml_path : str,
                    root_tag : str,
                    element_tag : str) -> int :
    """
        Appends layer description to xml file and if xml doesn't exist or is empty, 
        creates an xml file with required headers.

        Args:
        parameter_dictionary : Dictionary containing layer description.
        xml_path : Path where xml file will be stored / created.
        root_tag : Tag around which xml file will be wrapped.
        element_tag : Tag around which each element in dictionary will be wrapped.
    """
   
    if not os.path.exists(xml_path) :
        # Create base xml file.
        f = open(xml_path, "w")
        data = "<" + root_tag + ">" + "</" + root_tag + ">"
        f.write(data)
        f.close()
    layer_description = create_xml_tree(parameter_dictionary, element_tag)
    xml_file = ElementTree.parse(xml_path)
    root = xml_file.getroot()
    layer = root.makeelement(element_tag, parameter_dictionary)
    root.append(layer_description)
    xml_file.write(xml_path, encoding = "unicode")
    return 0

In [71]:
def iterate_over_layers(modules, xml_path, base_path, layer_index, debug : bool) -> int :
    """
        Parses model and generates csv and xml file which will be iterated by C++ translator.
    
        Args:
        modules : PyTorch model for which parameter csv and xml will be created.
        xml_path : Directory where xml with model config will be saved.
        base_path : Directory where csv will be stored.

        Returns 0 if weights are created else return 1.
    """
    for block in modules :
        for layer in block :
            layer_index += 1
            parameter_dict = extract_weights(layer, str(layer_index), base_path)
            create_xml_file(parameter_dict, xml_path, "model", "layer")
            if not os.path.exists(parameter_dict["weight_csv"]) and parameter_dict["has_weights"] == 1:
                print("Creating weights failed!")
                return 1, layer_index
            if debug :
                print("Weights created succesfully for ", parameter_dict["name"], " layer index :", layer_index)
    return 0, layer_index

In [72]:
def parse_model(model, xml_path, base_path, debug : bool) -> int :
    """
        Parses model and generates csv and xml file which will be iterated by C++ translator.
    
        Args:
        model : PyTorch model for which parameter csv and xml will be created.
        xml_path : Directory where xml with model config will be saved.
        base_path : Directory where csv will be stored.

        Returns 0 if weights are created else return 1.
    """
    layer_index = 0
    error, layer_index = iterate_over_layers(model.features, xml_path, base_path, layer_index, debug)
    if error :
        print("An error occured!")
        return 1
    print(layer_index)
    error, layer_index = iterate_over_layers(model.classifier, xml_path, base_path, layer_index, debug)
    if error :
        print("An error occured!")
        return 1
    print(layer_index)
    if debug :
        print("Model weights saved! Happy mlpack-translation.")
    return 0

In [73]:
parse_model(model, "model_config.xml", "./weights/darknet19/", True)

Weights created succesfully for  Convolution2D  layer index : 1
Weights created succesfully for  BatchNorm2D  layer index : 2
Weights created succesfully for  unknown_layer  layer index : 3
Weights created succesfully for  unknown_layer  layer index : 4
Weights created succesfully for  Convolution2D  layer index : 5
Weights created succesfully for  BatchNorm2D  layer index : 6
Weights created succesfully for  unknown_layer  layer index : 7
Weights created succesfully for  unknown_layer  layer index : 8
Weights created succesfully for  Convolution2D  layer index : 9
Weights created succesfully for  BatchNorm2D  layer index : 10
Weights created succesfully for  unknown_layer  layer index : 11
Weights created succesfully for  Convolution2D  layer index : 12
Weights created succesfully for  BatchNorm2D  layer index : 13
Weights created succesfully for  unknown_layer  layer index : 14
Weights created succesfully for  Convolution2D  layer index : 15
Weights created succesfully for  BatchNorm

0